In [6]:
import pandas as pd


train = pd.read_csv('../dataset/train.csv', index_col=0) # 学習用データ
test = pd.read_csv('../dataset/test.csv', index_col=0) # 学習用データ   # 評価用データ
data_file = pd.read_csv('Index.csv', index_col=0) # データの説明
sample_submit = pd.read_csv('../dataset/sample_submission.csv', index_col=0, header=None) # 応募用サンプルファイル
data_file.head(5)

,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,NGDPRPPPPC,NGDPPC,NGDPDPC,...,GGSB_NPGDP,GGXONLB,GGXONLB_NGDP,GGXWDN,GGXWDN_NGDP,GGXWDG,GGXWDG_NGDP,NGDP_FY,BCA,BCA_NGDPD
Year,,,,,,,,,,,,,,,,,,,,,
1980,"6,763.50",-0.257,"2,857.33","2,857.33","2,857.33",42.246,"29,713.75","32,016.17","12,552.94","12,552.94",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2,857.33",2.316,0.081
1981,"6,935.15",2.538,"3,207.03","3,207.03","3,207.03",46.243,"30,163.89","32,501.19","13,948.70","13,948.70",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,207.03",5.031,0.157
1982,"6,810.13",-1.803,"3,343.80","3,343.80","3,343.80",49.100,"29,337.82","31,611.11","14,404.99","14,404.99",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,343.80",-5.533,-0.165
1983,"7,122.30",4.584,"3,634.03","3,634.03","3,634.03",51.023,"30,405.15","32,761.15","15,513.68","15,513.68",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,634.03",-38.695,-1.065
1984,"7,637.70",7.236,"4,037.65","4,037.65","4,037.65",52.865,"32,321.06","34,825.51","17,086.44","17,086.44",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,037.65",-94.342,-2.337


In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


numeric_columns =['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc',
       'MIS_Status', 'Sector', 'ApprovalFY', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural']
train_numeric = train[numeric_columns]
# Convert categorical columns to one-hot encoded columns
categorical_columns = ['RevLineCr', 'LowDoc', 'UrbanRural']
train_numeric = pd.get_dummies(train, columns=categorical_columns)

# Convert currency columns to numeric
train_numeric['DisbursementGross'] = pd.to_numeric(train_numeric['DisbursementGross'], errors='coerce')
train_numeric['GrAppv'] = pd.to_numeric(train_numeric['GrAppv'], errors='coerce')
train_numeric['SBA_Appv'] = pd.to_numeric(train_numeric['SBA_Appv'], errors='coerce')

# Convert state columns to score of GDP
state_data = pd.read_csv('state_gdp.csv', index_col=0)
state_columns = ['State', 'BankState']
for col in state_columns:
    train_numeric[col] = train_numeric[col].map(state_data.set_index('State')['gdp'])

# Convert Year columns to score of economy and add Index to train_numeric
economy_data = pd.read_csv('Index.csv', index_col=0)
year_columns = ['ApprovalFY']
for col in year_columns:
    train_numeric[col] = train_numeric[col].map(economy_data.set_index('Year')['index'])
    train_numeric['Index'] = data_file['Index']




# 数値列だけを選択
numeric_columns = train_numeric.select_dtypes(include=['number']).columns
# 数値列だけを対象に平均計算
train_numeric[numeric_columns] = train_numeric[numeric_columns].fillna(train_numeric[numeric_columns].mean())

# 残りの文字列列に対しては、NaN をそのままにするか、適切な処理を行う
# 例えば、文字列列に対しては欠損値を特定の文字列で置き換えたりすることが考えられます

X = train_numeric.drop(['DisbursementDate', 'ApprovalDate', 'City','MIS_Status'], axis=1)
y = train_numeric['MIS_Status']                # 目的変数

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.dtypes)

# LightGBMモデルの設定
lgbm = lgb.LGBMClassifier()

# モデルの訓練
lgbm.fit(X_train, y_train)

# モデルの評価
y_pred_categorical = lgbm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred_categorical)}')


Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
Sector                 int64
ApprovalFY             int64
State                  int64
BankState            float64
DisbursementGross    float64
GrAppv               float64
SBA_Appv             float64
RevLineCr_0             bool
RevLineCr_N             bool
RevLineCr_T             bool
RevLineCr_Y             bool
LowDoc_0                bool
LowDoc_A                bool
LowDoc_C                bool
LowDoc_N                bool
LowDoc_S                bool
LowDoc_Y                bool
UrbanRural_0            bool
UrbanRural_1            bool
UrbanRural_2            bool
dtype: object
[LightGBM] [Info] Number of positive: 30192, number of negative: 3653
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003854 seconds.
You can set `force_row_wise=true` to remove the overhead.


In [5]:
from sklearn.metrics import f1_score
  
mean_f1 = f1_score(y_test, y_pred_categorical, average='macro')
print(f'Mean F1 Score: {mean_f1}')

Mean F1 Score: 0.6344638947781297
